In [37]:
# Importamos la libreria request
import requests

def translate(text, source='en', target='es'):
    params = {'sl': source, 'tl': target, 'q': text}
    headers = {"Charset":"UTF-8","User-Agent":"AndroidTranslate/5.3.0.RC02.130475354-53000263 5.1 phone TRANSLATE_OPM5_TEST_1"}
    url = "https://translate.google.com/translate_a/single?client=at&dt=t&dt=ld&dt=qca&dt=rm&dt=bd&dj=1&hl=es-ES&ie=UTF-8&oe=UTF-8&inputm=2&otf=2&iid=1dd3b944-fa62-4b55-b330-74909a99969e"
    response = requests.post(url, data=params, headers=headers)
    if response.status_code == 200: 
        return response.json()
    else:
        print("Ocurrió un error", text)
        return "Ocurrió un error"



In [ ]:
#Interfas para usuario
import json

count = 0
while count < 1:
    count += 1
    text = input("Ingrese un texto para traducir: ")
    respuesta = json.dumps(translate(text),ensure_ascii=False, indent = 2)
    print(respuesta)

In [20]:
import json

with open("wordsList.json", "r") as f:
    wordsList = json.loads(f.read())
    dicList = list(map(translate, wordsList))

dictData = {}
for x in dicList:
    ID = x['sentences'][0]['orig']
    dictData[ID] = x
    
with open("googleDict.json", "w") as outfile:
    json.dump(dictData, outfile, ensure_ascii = False)

In [38]:
import json

def transEgs(meaning):
    ejemplos = []
    for eg in meaning["examples"]:
        ejemplos.append(translate(eg)["sentences"][0]["trans"])
    meaning["ejemplos"] = ejemplos
    return meaning


with open("wikiData.json", "r") as f:
    wikiData = json.loads(f.read())
    for word in wikiData:
        wikiData[word]["meanings"] = list(map(transEgs, wikiData[word]["meanings"]))
        
with open("wikiData_es.json", "w") as f:
    json.dump(wikiData, f, ensure_ascii = False)

In [40]:
import json
import re

with open("wikiData_es.json", "r") as wikiData, open("googleDict.json", 'r') as googleDict:
    wikiData = json.loads(wikiData.read())
    googleDict = json.loads(googleDict.read())

    def delSomeEgs(meaning, translations):
        ej_list = meaning["ejemplos"]
        eg_list = meaning["examples"]
        newEg_list = []
        newEj_list = []
        trans_terms = []
        count = 0
        for ejemplo in ej_list:
            include = False
            for trans in translations:
                if include:
                    break
                if re.search(rf"[\s\"]{trans} ",ejemplo):
                    newEg_list.append(eg_list[count])
                    newEj_list.append(ejemplo)
                    trans_terms.append(trans) 
            count += 1
        meaning["examples"] = newEg_list
        meaning["ejemplos"] = newEj_list
        meaning["trans_terms"] = trans_terms
        return meaning

    
    classes = {
    'article': 'artículo',
    'preposition': 'preposición',
    'adverb': 'adverbio',
    'adjective': 'adjetivo',
    'noun': 'sustantivo',
    'conjunction': 'conjunción',
    'determiner': 'adjetivo',
    'pronoun': 'pronombre',
    'verb': 'verbo',
    'proper noun': 'sustantivo',
    'numeral': 'sustantivo',
    'symbol': 'sustantivo'
    }

    for word in wikiData:
        word_meanings = []
        for meaning in wikiData[word]["meanings"]:
            pos = meaning["class"]
            if pos == "interjection":
                continue
            pos = classes[pos]
            if "dict" in googleDict[word]:
                for entry in googleDict[word]["dict"]:
                    if entry["pos"] == pos:
                        word_meanings.append(delSomeEgs(meaning, entry["terms"]))                     
        wikiData[word]["meanings"] = word_meanings


with open("ownDict.json", "w") as f:
    json.dump(wikiData, f, ensure_ascii = False)

In [39]:

with open("wikiData.json", "r") as f:
    wikiData = json.loads(f.read())
    classes = []
    for word in wikiData:
        for meaning in wikiData[word]["meanings"]:
            wordClass = meaning["class"]
            if wordClass not in classes:
                classes.append(wordClass)

print(classes)

['article', 'preposition', 'symbol', 'adverb', 'adjective', 'noun', 'conjunction', 'determiner', 'pronoun', 'verb', 'interjection', 'proper noun', 'numeral']


In [13]:

with open("googleDict.json", "r") as f:
    Data = json.loads(f.read())
    classes = []
    for word in Data:
        if "dict" in Data[word]:
            for meaning in Data[word]["dict"]:
                wordClass = meaning["pos"]
                if wordClass not in classes:
                    classes.append(wordClass)

print(classes)

['artículo', 'adjetivo', 'preposición', 'adverbio', 'sustantivo', 'conjunción', 'pronombre', 'verbo', '', 'partícula']
